In [1]:
include(joinpath(Pkg.dir("Crispulator"), "src", "simulation", "load.jl"))
include(joinpath(Pkg.dir("Crispulator"), "src", "simulation", "common.jl"))
include(joinpath(Pkg.dir("Crispulator"), "src", "simulation", "utils.jl"))
include(joinpath(Pkg.dir("Crispulator"), "src", "simulation", "library.jl"))
include(joinpath(Pkg.dir("Crispulator"), "src", "simulation", "designs.jl"))
include(joinpath(Pkg.dir("Crispulator"), "src", "simulation", "processing.jl"))
include(joinpath(Pkg.dir("Crispulator"), "src", "simulation", "selection.jl"))
include(joinpath(Pkg.dir("Crispulator"), "src", "simulation", "sequencing.jl"))
include(joinpath(Pkg.dir("Crispulator"), "src", "simulation", "transfection.jl"))

INFO: Loading simulation framework


transfect (generic function with 2 methods)

In [60]:
pheno_prob = 0.4
growth_param = GrowthScreen()
growth_param.num_genes = 10
growth_param.coverage = 3
growth_param.representation = 1000
growth_param.seq_depth = 1000
growth_param.noise = 0.1
max_phenotype_dists = Dict{Symbol, Tuple{Float64, Sampleable}}(
    :inactive => (1-0.05-pheno_prob, Delta(0.0)),
    :negcontrol => (0.05, Delta(0.0)),
    :increasing => (pheno_prob/2, TruncatedNormal(0.1, 0.1, 0.025, 1)),
    :decreasing => (pheno_prob/2, TruncatedNormal(-0.55, 0.2, -1, -0.1))
)

lib = Library(max_phenotype_dists, CRISPRn())
guides, guide_freqs_dist = construct_library(growth_param, lib);

In [64]:
cells, cell_phenotypes = transfect(growth_param, lib, guides, guide_freqs_dist)
bin_cells = select(growth_param, cells, cell_phenotypes, guides)
freqs = counts_to_freqs(bin_cells, length(guides))

seq_depths = Dict{Symbol, Int}()
for binname in keys(bin_cells)
    seq_depths[binname] = rand(Poisson(growth_param.seq_depth))
end

raw_data = sequencing(seq_depths, guides, freqs)
bc_counts, genes = differences_between_bins(raw_data);

In [65]:
bc_counts

,gene,knockdown,theo_phenotype,obs_phenotype,behavior,class,initial_freq,counts_bin1,barcodeid,freqs_bin1,rel_freqs_bin1,freqs_bin2,counts_bin2,rel_freqs_bin2,log2fc_bin2
1,1,1.0,0.0,-Inf,sigmoidal,negcontrol,0.049185861795075454,1487.5,1,0.04945968412302577,1.3382816014394963,0.03392534122677191,1031.5,1.0563236047107016,-0.341329848055985
2,1,1.0,0.0,-Inf,sigmoidal,negcontrol,0.03762907069102462,1111.5,2,0.0369576059850374,1.0,0.03211642821904292,976.5,1.0,0.0
3,1,1.0,0.0,-Inf,sigmoidal,negcontrol,0.02635027799841144,778.5,3,0.025885286783042395,0.7004048582995952,0.01984870909389903,603.5,0.6180235535074246,-0.18052726918381723
4,2,1.0,0.0,-Inf,linear,inactive,0.02789912629070691,783.5,4,0.02605153782211139,0.7049032838506524,0.021920736720934055,666.5,0.6825396825396826,-0.04651240024340903
5,2,1.0,0.0,-Inf,linear,inactive,0.04841143764892772,1482.5,5,0.04929343308395678,1.3337831758884393,0.0356684755796744,1084.5,1.110599078341014,-0.26418605262089373
6,2,1.0,0.0,-Inf,linear,inactive,0.024463860206513106,727.5,6,0.024189526184538655,0.6545209176788125,0.019947377076138792,606.5,0.6210957501280082,-0.07562360353493768
7,3,1.0,0.0,-Inf,linear,inactive,0.04056791104050834,1210.5,7,0.04024937655860349,1.0890688259109313,0.030044400592007893,913.5,0.935483870967742,-0.21931044623124982
8,3,1.0,0.0,-Inf,linear,inactive,0.011834789515488484,360.5,8,0.01198669991687448,0.32433648223121914,0.008765005755632297,266.5,0.2729134664618536,-0.24904772734144828
9,3,1.0,0.0,-Inf,linear,inactive,0.02972597299444003,917.5,9,0.030507065669160432,0.8254610886189834,0.018270021378062818,555.5,0.568868407578085,-0.5371052472012849
10,4,1.0,0.0,-Inf,linear,inactive,0.014912629070691024,435.5,10,0.014480465502909393,0.39181286549707606,0.009784574905443183,297.5,0.3046594982078853,-0.3629670512633606
